In [1]:
# libraries
library(Seurat)
library(tidyverse)
library(igraph)
require(circlize)
library(R.utils)
library(data.table) #to read gz file

Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2      ✔ purrr   0.3.5 
✔ tibble  3.2.1      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘igraph’


The following objects are masked from ‘package:dplyr’:

    as_data_frame, groups, union


The following objects are masked from ‘package:purrr’:

    compose, simplify


The following object is masked from ‘package:tidyr’:

    crossing


The following object is masked from ‘package:tibble’:

    as_data_frame


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: circlize

circlize version 0.4.15
CRAN page: https://cran.r-projec

In [3]:
input_dir <- "../../../../results/data_preprocessing/Lasry/"

In [4]:
# # load counts
# print("load counts")

counts <- fread(paste0(input_dir,"counts_corr.csv.gz"), header = TRUE,check.names=FALSE)
counts <- as.data.frame(counts)
rownames(counts) <- counts$gene_symbol
counts <- counts[,-1]

In [5]:
# load cell annotation
print("load cell annotation")
anno_cells <- read.table(paste0(input_dir,"anno_cells_corr.txt")
                         ,sep = "\t"
                         ,row.names = 1
                         ,header = TRUE
                         ,check.names=FALSE
                         )
# print(str(anno_cells))

[1] "load cell annotation"


In [6]:
#set rownames of annotation to cell_ids
rownames(anno_cells) <- anno_cells$cell

#set colnames of counts to cell_ids
colnames(counts) <- rownames(anno_cells)

In [46]:
avg_expression <- rowMeans(counts)

In [47]:
gene_names <- rownames(counts)

In [48]:
avg_expr_table <- data.frame(Gene = gene_names, AverageExpression = avg_expression)

In [49]:
case <- anno_cells$health_status == "AML"
case_idx <- rownames(anno_cells)[case]

In [50]:
avg_expression_case <- rowMeans(counts[,case_idx])
# gene_names <- rownames(counts[,case_idx])
# avg_expr_table_case <- data.frame(Gene = gene_names, AverageExpression = avg_expression)

In [51]:
control <- anno_cells$health_status == "healthy"
control_idx <- rownames(anno_cells)[control]

In [52]:
avg_expression_control <- rowMeans(counts[,control_idx])

In [53]:
avg_expr_table$control <- avg_expression_control

In [55]:
avg_expr_table$case <- avg_expression_case

In [6]:
#create a Seurat object
srt=CreateSeuratObject(counts=counts, meta.data=anno_cells)

In [7]:
#peek into the number of cells for case/control
srt@meta.data$health_status %>% table()

.
    AML healthy 
  21311   25391 

In [8]:
#peek into the number of cell types
srt@meta.data$cell_type %>% table()

.
    B    DC   Ery  Gran  HSPC  Mono    NK     T 
 4765  1634  1674  2332  3169 18004  3078 12046 

In [9]:
#set the indent to cell_type
Idents(srt) <- "cell_type"

In [10]:
# initialize empty vector for storing DEGs
DEGs <- c()

# iterate over each unique cell type 
for (cell_type in unique(srt@meta.data$cell_type)) {
  
  # subset Seurat object to only include cells of current cell type
  seurat_obj_receiver <- subset(srt, idents = cell_type)
  
  # set cell identity using the "health_status" feature
  seurat_obj_receiver <- SetIdent(seurat_obj_receiver, value = seurat_obj_receiver[["health_status"]])
  
  # specify the two conditions to compare
  condition_oi <- "AML"
  condition_reference <- "healthy" 
  
#   find differentially expressed genes between the two conditions
  DE_table_receiver <- FindMarkers(object = seurat_obj_receiver, 
                                   slot = "counts",
                                   ident.1 = condition_oi, 
                                   ident.2 = condition_reference, 
                                   min.pct = 0.10) %>%
    # convert row names to a separate "gene" column
    rownames_to_column("gene")
    
#     DE_table_receiver <- FindMarkers(object = seurat_obj_receiver, 
#                                    slot = "data",
#                                    ident.1 = condition_oi, 
#                                    ident.2 = condition_reference, 
#                                    min.pct = -Inf,
#                                    thresh.use = 0,
#                                    logfc.threshold = -Inf,
#                                    min.cells.feature = 1, min.cells.group = 1) %>% rownames_to_column("gene")
    
    
    
  
  # add cell type information to the DEG table
  DE_table_receiver <- data.frame(cluster = cell_type, DE_table_receiver)
  
  # filter DEGs based on statistical significance and fold change threshold
#   DE_table_receiver <- DE_table_receiver %>% 
#     filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)
  
  # print cell type and number of DEGs found
  print(cell_type)
  print("number of genes in the cell type")
  print(length(row.names(seurat_obj_receiver@assays$RNA@data)))
  print("number of genes output from DE")
  print(nrow(DE_table_receiver))
  
  # append DEGs to the vector of all DEGs
  DEGs <- rbind(DEGs, DE_table_receiver)
}

[1] "Mono"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 178
[1] "Gran"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 103
[1] "T"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 48
[1] "NK"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 164
[1] "B"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 74
[1] "HSPC"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 90
[1] "Ery"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 645
[1] "DC"
[1] "number of genes in the cell type"
[1] 15770
[1] "number of genes output from DE"
[1] 56


In [57]:
write.csv(DEGs, "DEG_table.csv")

In [59]:
write.csv(DEGs, "DEG_significant_table.csv")